In [8]:
import duckdb

In [15]:
cn = duckdb.connect("order_repo.db")

In [16]:
sql = """
    select *
    from order_event_dtl
    where order_id = 3
        and event_type = 99
"""
cn.sql(sql).show()

┌──────────┬────────────┬────────────┬─────────────────────┐
│ order_id │ event_type │ event_desc │      event_ts       │
│  int32   │   int32    │  varchar   │      timestamp      │
├──────────┼────────────┼────────────┼─────────────────────┤
│        3 │         99 │ exception  │ 2024-01-06 08:12:02 │
│        3 │         99 │ exception  │ 2024-01-06 10:08:11 │
└──────────┴────────────┴────────────┴─────────────────────┘



In [17]:
sql = """
    select *
    from order_event_ex
    where order_id = 3
"""
cn.sql(sql).show()

┌──────────┬─────────────────────┬───────────────────────────────┐
│ order_id │      except_ts      │            ex_desc            │
│  int32   │      timestamp      │            varchar            │
├──────────┼─────────────────────┼───────────────────────────────┤
│        3 │ 2024-01-06 08:15:05 │ order lost - creating new one │
│        3 │ 2024-01-06 10:10:16 │ package fell off conveyor     │
└──────────┴─────────────────────┴───────────────────────────────┘



In [29]:
"""
to do:
    harder this query
    explain the load shell script and chmod stuff
    publish
"""

sql = """
    with exd as (
        select dtl.order_id, dtl.event_ts, ex.except_ts, ex.ex_desc
        from order_event_dtl as dtl
            inner join order_event_ex as ex
                on dtl.order_id = ex.order_id
        where dtl.order_id = 3
            and dtl.event_type = 99
            and datediff('minute', dtl.event_ts, ex.except_ts) between 0 and 5
    )

    select hdr.order_id
        ,exd.*
    from order_hdr as hdr
        inner join order_event_dtl as ev
            on hdr.order_id = ev.order_id
        left join exd 
            on ev.order_id = exd.order_id and ev.event_ts = exd.event_ts

"""
cn.sql(sql).show()

┌──────────┬──────────┬─────────────────────┬─────────────────────┬───────────────────────────────┐
│ order_id │ order_id │      event_ts       │      except_ts      │            ex_desc            │
│  int32   │  int32   │      timestamp      │      timestamp      │            varchar            │
├──────────┼──────────┼─────────────────────┼─────────────────────┼───────────────────────────────┤
│        3 │        3 │ 2024-01-06 08:12:02 │ 2024-01-06 08:15:05 │ order lost - creating new one │
│        3 │        3 │ 2024-01-06 10:08:11 │ 2024-01-06 10:10:16 │ package fell off conveyor     │
│        1 │     NULL │ NULL                │ NULL                │ NULL                          │
│        1 │     NULL │ NULL                │ NULL                │ NULL                          │
│        1 │     NULL │ NULL                │ NULL                │ NULL                          │
│        1 │     NULL │ NULL                │ NULL                │ NULL                          │


In [6]:
sql = """
    SELECT h.order_id, ev.event_ts, ev.event_desc, ev.event_type
        ,datediff('second', h.crt_ts, ev.event_ts) as ts_diff
    FROM order_hdr as h
        inner join order_event as ev
            on h.order_id = ev.order_id
    where h.order_id = 1
    qualify 1 = rank() over(partition by ev.order_id, ev.event_type order by abs(datediff('second', h.crt_ts, ev.event_ts)))
    order by event_ts
"""
cn.sql(sql).show()

┌──────────┬─────────────────────┬─────────────────────────┬────────────┬─────────┐
│ order_id │      event_ts       │       event_desc        │ event_type │ ts_diff │
│  int32   │      timestamp      │         varchar         │   int32    │  int64  │
├──────────┼─────────────────────┼─────────────────────────┼────────────┼─────────┤
│        1 │ 2024-01-01 08:05:32 │ order_create            │          1 │       9 │
│        1 │ 2024-01-01 09:13:44 │ order_pack              │          2 │    4101 │
│        1 │ 2024-01-01 09:15:12 │ conveyor_load           │          3 │    4189 │
│        1 │ 2024-01-01 09:35:03 │ exception_missing_label │         99 │    5380 │
│        1 │ 2024-01-01 18:34:16 │ order_shipped           │          4 │   37733 │
│        1 │ 2024-01-03 07:09:33 │ order_delivered         │          5 │  169450 │
└──────────┴─────────────────────┴─────────────────────────┴────────────┴─────────┘



In [14]:
cn.close()